In [1]:
import numpy as np
import pandas as pd

# Loading the data

Here, I use the preprocessing pipeline that Pedro developed to start playing with the unsupervised NN model.

In [2]:
data = pd.read_csv('./raw_data/tracks_features.csv')

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import RobustScaler

def preprocess_features(df):
    def preproc_pipeline():

        features_num = [
            "danceability",
            "energy",
            "acousticness",
            "instrumentalness",
            "valence",
            "tempo",
            "duration_ms",
            "loudness",
            "speechiness",
            "liveness"
            ]

        #features_ohe = [
        #    "key",
        #    "time_signature",
            #"explicit"
         #  ]

        preproc_pipe = make_column_transformer(
            (RobustScaler(),features_num),
          #  (OneHotEncoder(sparse_output = False),features_ohe),
            remainder="passthrough"
            )

        return preproc_pipe

    pipeline = preproc_pipeline()

    df_preprocessed = pipeline.fit_transform(df)

    df_preprocessed = pd.DataFrame(df_preprocessed)

    return df_preprocessed

In [4]:
data_num = data.select_dtypes(exclude = ['object']).drop(columns = ['track_number', 'disc_number', 'key'])

data_preproc = preprocess_features(data_num)

In [20]:
print(data.columns)
print(data_num.columns)
data_preproc.head()

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')
Index(['explicit', 'danceability', 'energy', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'time_signature', 'year'],
      dtype='object')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.111913,0.883268,-0.440734,-0.011223,0.220751,0.027447,-0.127123,0.514466,0.755376,1.558704,False,1,4.0,1999
1,0.353791,0.842412,-0.456765,-0.01114,0.189845,-0.303452,-0.162318,0.471711,3.854839,0.202429,True,1,4.0,1999
2,-0.67148,0.867704,-0.444013,-0.011235,-0.072848,0.76812,0.66715,0.511538,11.784946,-0.020243,False,1,4.0,1999
3,-0.220217,0.861868,-0.274472,-0.011233,0.377483,-0.464598,-0.09574,0.46398,5.172043,-0.026991,True,0,4.0,1999
4,-0.270758,0.787938,-0.470464,0.1348,0.300221,0.240347,-0.167687,0.358674,0.685484,-0.311066,False,1,4.0,1999


## PCA

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=7)
data_pca = pca.fit_transform(data_preproc)


In [8]:
data_pca = pca.transform(data_preproc)

: 

In [6]:
pd.DataFrame(pca.components_.T, index=data_num.columns)

,0,1,2,3,4,5,6
explicit,-0.004017,0.048808,-0.099829,0.087162,0.343384,-0.437755,-0.531304
danceability,-0.006901,0.026602,-0.046271,0.271371,0.307489,0.068854,0.291758
energy,0.005910,-0.013599,0.024487,-0.173969,-0.239403,-0.046151,-0.225017
loudness,0.000391,-0.030930,0.060742,-0.117913,-0.145664,0.121201,-0.018347
mode,0.002606,0.023819,-0.116844,0.146787,0.265580,-0.203763,-0.419898
speechiness,-0.002387,0.002782,-0.031949,0.129998,0.210650,0.856217,-0.429518
acousticness,0.001102,-0.047539,0.978193,0.063788,0.167084,-0.044229,-0.059309
instrumentalness,-0.011761,0.019634,-0.085634,0.371319,0.505364,-0.002036,0.414531
liveness,-0.016100,0.991934,0.056940,-0.098562,0.003900,0.028798,0.024511
valence,0.001461,0.086933,0.035432,0.824513,-0.540472,-0.053270,-0.112298


## Unsupervised NN

In [3]:
song_name = "Busy - (For Me)"
artist = "Aurea"

data.index[(data.name == song_name ) & (data.artists.str.strip('[]').str.strip("'") == artist)][0]
#data[(data.name == "D'Evils" )]['artists'].str.strip('[]').str.strip("'")

# this whole line after .artists might not be needed, or there might be a neater way to do it, but it works

IndexError: index 0 is out of bounds for axis 0 with size 0

In [37]:
data.iloc[480708]

id                                             7jszJecKQj9kVYjtFXboRR
name                                                         Bell Boy
album               Bringing Out The Dead - Music From The Motion ...
album_id                                       30yEwgpAvxeVE7Kpn8UPd4
artists                                                   ['The Who']
artist_ids                                 ['67ea9eGLXYMsO2eYQRui3w']
track_number                                                       12
disc_number                                                         1
explicit                                                        False
danceability                                                    0.574
energy                                                          0.821
key                                                                 8
loudness                                                       -8.249
mode                                                                1
speechiness         

In [7]:
from sklearn.neighbors import NearestNeighbors

song_knn = NearestNeighbors(n_neighbors=10000).fit(data_pca)

In [44]:
song_i = song_knn.kneighbors(data_pca[480708].reshape(1,-1), return_distance=False)

In [45]:
data.iloc[song_i[0]]

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
480708,23NPGXlSaIqWzvxIRhM2oG,Under the Bridge,Greatest Hits,53tvjWbVNZKd3CvpENkzOC,['Red Hot Chili Peppers'],['0L8ExT028jH3ddEcZwqJJ5'],1,1,False,0.554,...,0.0457,0.01680,0.000534,0.136,0.513,84.275,265507,4.0,2003,2003-11-18
496534,4zkzkEQGjfzmq0tVzpsMdM,Under the Bridge,Greatest Hits,6kLm0YrEESYq7yszohzkA1,['Red Hot Chili Peppers'],['0L8ExT028jH3ddEcZwqJJ5'],1,1,False,0.554,...,0.0457,0.01680,0.000534,0.136,0.513,84.275,265507,4.0,2003,2003-11-18
89125,39Lgjxyxc9lyzYsHOPLgoD,Earthbound,Universe,3WTRtBAE8mLaxYepMZtQHU,['Wideband Network'],['4KngAWJlEwlm3oFbuyOQ69'],2,1,False,0.554,...,0.0392,0.07250,0.000000,0.145,0.595,88.023,268053,4.0,2003,2003-11-09
1060340,65KosTcT5wLK03rNTV30Mr,Ready...dub!,Dub Confrontation,0R8mBEMRx0n1e3irkkCbAQ,"['Warsaw Poland Bros.', 'Stucky']","['1GhwxIv0SfMGqLFatev2B4', '23N3acndLUkdQpwhQi...",10,1,False,0.652,...,0.0430,0.03560,0.157000,0.117,0.446,79.168,255320,4.0,2003,2003
144026,4OVqt4SiJoXiTRNH8TcSIf,Jesus Is Coming Back,One More River To Cross,44qesR56w0JIU6b4ymfqRT,['Zion Jubilees'],['3UG5PWB6OlaK96Xk8dlCzi'],2,1,False,0.529,...,0.0546,0.10400,0.000000,0.136,0.653,76.101,260000,4.0,2003,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52132,0Q4Ekl18A9ORQdXl4m1zDT,Jenny's Gone Away,Heading For Home,7AhuU9cQZjOiHkEYmDL5v3,['Peggy Seeger'],['0gXQn7UrpopDb2TIkmIjoU'],9,1,False,0.617,...,0.0316,0.85200,0.000000,0.104,0.495,72.618,169067,4.0,2003,2003-10-07
29894,3GHcf34wxOrl6Y07Kcf6Lr,I'm No Stranger to the Rain,RCA Country Legends,6O2uWqGjs48lWQUtE5LDtw,['Keith Whitley'],['0bc7dhY6YXL35m1G0jVwuW'],10,1,False,0.659,...,0.0227,0.30200,0.313000,0.101,0.538,88.125,214507,4.0,2002,2002-03-04
861042,46HH31imhPefdQceH6kmUC,Paper Angels,I Believe In Fairy Tales,4GU39ynD11MP4GzVaViH3D,['Sky Salt'],['5E681TQIwFtC3sYVj65uUb'],2,1,False,0.507,...,0.0290,0.00946,0.000003,0.110,0.467,109.093,210320,3.0,2004,2004-01-01
218373,5MCn0hsgDGAb2Zey4Af1o0,Zamba De La Candelaria - Remastered 2003,Los Chalchaleros (1958),6Kwwiyh4C0kddn9VAt6U5h,['Los Chalchaleros'],['0t8lKcdJV0dEicuocOteOw'],9,1,False,0.674,...,0.0684,0.88000,0.000000,0.155,0.242,80.077,210387,3.0,2004,2004-11-16


: 